<a href="https://colab.research.google.com/github/mounirhomrani2-prog/tts-bambara-project/blob/ovoice-feature-coqui-tts/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installation Libraries**

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!apt-get update -qq 2>/dev/null || true
!apt-get install -y build-essential espeak espeak-data libespeak1 libespeak-dev
print("📦 Installation de CoquiTTS...")
!pip install coqui-tts
!apt-get update -qq 2>/dev/null || true
!apt-get install -y build-essential espeak espeak-data libespeak1 libespeak-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
The following additional packages will be installed:
  libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak-dev libespeak1 libportaudio2 libsonic0
0 upgraded, 6 newly installed, 0 to remove and 90 not upgraded.
Need to get 1,575 kB of archives.
After this operation, 3,802 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak amd64 1.4

In [4]:
!pip install torch torchaudio
!pip install numpy scipy librosa soundfile

In [5]:
# Installation de Coqui TTS et dépendances
!pip install -q pandas pyarrow soundfile librosa

# Vérifier l'installation
import TTS
print(f"✅ TTS version : {TTS.__version__}")

# Imports nécessaires
import os
import json
import soundfile as sf
from pathlib import Path
import shutil

✅ TTS version : 0.27.3


# **Verification of the installed environment**

In [6]:
# Re-vérification après installation
print("🔧 Re-vérification après installation...")

try:
    import TTS
    print(f"✅ CoquiTTS: {TTS.__version__}")

    # Test d'import des modules principaux
    from TTS.tts.configs.shared_configs import BaseDatasetConfig
    from TTS.tts.configs.vits_config import VitsConfig
    print("✅ Modules TTS importés avec succès")

except Exception as e:
    print(f"❌ Problème avec CoquiTTS: {e}")

# Vérification de PyTorch (important pour GPU)
import torch
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🔥 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  Mode CPU seulement")

print("\n" + "="*50)

🔧 Re-vérification après installation...
✅ CoquiTTS: 0.27.3
✅ Modules TTS importés avec succès
🔥 PyTorch: 2.9.0+cu126
🔥 CUDA disponible: True
🔥 GPU: NVIDIA A100-SXM4-80GB



# **Defines name, formatter and path of the dataset**

In [7]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)


# **Copy dataset from Drive**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from google.colab import drive
import shutil
import os

# Monter Google Drive (déjà monté)
# drive.mount('/content/drive')

print("=" * 50)
print("EXPLORATION DE GOOGLE DRIVE")
print("=" * 50)

# Lister le contenu principal
print("\n📁 Contenu de MyDrive :")
!ls -lh "/content/drive/MyDrive/Bambara_TTS/afvoices_permanent/human-corrected"

EXPLORATION DE GOOGLE DRIVE

📁 Contenu de MyDrive :
total 16G
-rw------- 1 root root 366M Jan 15 09:51 test-00000-of-00004.parquet
-rw------- 1 root root 357M Jan 15 09:51 test-00001-of-00004.parquet
-rw------- 1 root root    0 Jan 15 10:50 test-00002-of-00004.parquet
-rw------- 1 root root 357M Jan 15 09:51 test-00003-of-00004.parquet
-rw------- 1 root root 430M Jan 15 09:51 train-00001-of-00126.parquet
-rw------- 1 root root 428M Jan 15 09:51 train-00002-of-00126.parquet
-rw------- 1 root root 441M Jan 15 09:49 train-00003-of-00126.parquet
-rw------- 1 root root    0 Jan 15 10:50 train-00004-of-00126.parquet
-rw------- 1 root root 433M Jan 15 09:50 train-00008-of-00126.parquet
-rw------- 1 root root 431M Jan 15 09:51 train-00010-of-00126.parquet
-rw------- 1 root root 449M Jan 15 09:43 train-00014-of-00126.parquet
-rw------- 1 root root    0 Jan 15 10:50 train-00015-of-00126.parquet
-rw------- 1 root root 444M Jan 15 09:47 train-00017-of-00126.parquet
-rw------- 1 root root 432M Jan 

In [10]:
import shutil
import os

print("=" * 50)
print("COPIE DES DONNÉES DEPUIS GOOGLE DRIVE")
print("=" * 50)

# Chemin source sur Drive
source_path = "/content/drive/MyDrive/Bambara_TTS/afvoices_permanent"

# Chemin de destination dans Colab
destination_path = "/content/afvoices"

# Vérifier si la source existe
if os.path.exists(source_path):
    print(f"✅ Source trouvée : {source_path}")

    # Supprimer la destination si elle existe déjà
    if os.path.exists(destination_path):
        print(f"🗑️ Suppression de l'ancien dossier...")
        shutil.rmtree(destination_path)

    # Copier les données
    print(f"📋 Copie en cours...")
    shutil.copytree(source_path, destination_path)

    print(f"✅ Données copiées avec succès vers : {destination_path}")

    # Vérifier le contenu
    print("\n📁 Contenu copié :")
    !ls -lh /content/afvoices/

else:
    print(f"❌ Source introuvable : {source_path}")
    print("\n📁 Vérifiez le chemin. Contenu de Bambara_TTS :")
    !ls -lh "/content/drive/MyDrive/Bambara_TTS/"


COPIE DES DONNÉES DEPUIS GOOGLE DRIVE
✅ Source trouvée : /content/drive/MyDrive/Bambara_TTS/afvoices_permanent
📋 Copie en cours...
✅ Données copiées avec succès vers : /content/afvoices

📁 Contenu copié :
total 16K
drwx------ 3 root root 4.0K Jan 15 10:47 human-corrected
-rw------- 1 root root 8.6K Jan 15 09:43 README.md


# **Verification of dataset**

In [11]:
import os

# Vérifier le contenu du dossier
!ls -la afvoices/

# Explorer la structure
for root, dirs, files in os.walk('afvoices'):
    level = root.replace('afvoices', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Afficher les 5 premiers fichiers
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... et {len(files)-5} autres fichiers')

total 28
drwx------ 4 root root 4096 Jan 15 10:47 .
drwxr-xr-x 1 root root 4096 Jan 16 13:18 ..
drwx------ 9 root root 4096 Jan 15 10:50 .git
-rw------- 1 root root    0 Jan 15 10:50 .gitattributes
drwx------ 3 root root 4096 Jan 15 10:47 human-corrected
-rw------- 1 root root 8806 Jan 15 09:43 README.md
afvoices/
  .gitattributes
  README.md
  .git/
    config
    packed-refs
    HEAD
    branches/
    hooks/
      commit-msg.sample
      prepare-commit-msg.sample
      pre-push.sample
      post-commit
      post-merge
      ... et 12 autres fichiers
    refs/
      tags/
      remotes/
        origin/
          HEAD
      heads/
        main
    logs/
      refs/
        remotes/
          origin/
        heads/
    info/
      exclude
    objects/
      pack/
        pack-443f473ed287ef39b1cf64291466195165d4c3d2.idx
        pack-443f473ed287ef39b1cf64291466195165d4c3d2.pack
      info/
    lfs/
      tmp/
      incomplete/
      objects/
        71/
          71/
            7171a5

In [12]:
import pandas as pd
import os
from pathlib import Path
import glob

# Créer les dossiers nécessaires
output_dir = "tts_dataset"
wavs_dir = os.path.join(output_dir, "wavs")
os.makedirs(wavs_dir, exist_ok=True)

print("=" * 50)
print("VÉRIFICATION DES FICHIERS PARQUET")
print("=" * 50)

# Charger tous les fichiers parquet du dossier human-corrected
hc_files = glob.glob('afvoices/human-corrected/*.parquet')
print(f"\n📊 Nombre de fichiers trouvés : {len(hc_files)}\n")

# Vérifier la taille de chaque fichier et charger uniquement les valides
all_data = []
valid_files = 0
invalid_files = 0

for file in hc_files:
    file_size = os.path.getsize(file)

    if file_size > 0:
        try:
            df = pd.read_parquet(file)
            all_data.append(df)
            valid_files += 1
            print(f"✅ {os.path.basename(file)} - {file_size} bytes - {len(df)} lignes")
        except Exception as e:
            invalid_files += 1
            print(f"❌ {os.path.basename(file)} - Erreur : {str(e)[:50]}")
    else:
        invalid_files += 1
        print(f"⚠️ {os.path.basename(file)} - Fichier vide (0 bytes)")

print("\n" + "=" * 50)
print("RÉSUMÉ")
print("=" * 50)
print(f"✅ Fichiers valides : {valid_files}")
print(f"❌ Fichiers invalides/vides : {invalid_files}")

if all_data:
    df_combined = pd.concat(all_data, ignore_index=True)
    print(f"📏 Total d'échantillons chargés : {len(df_combined)}")
    print(f"\nColonnes : {list(df_combined.columns)}")
    print("\nAperçu :")
    print(df_combined.head(3))
else:
    print("⚠️ Aucune donnée valide trouvée !")


VÉRIFICATION DES FICHIERS PARQUET

📊 Nombre de fichiers trouvés : 55

⚠️ train-00023-of-00126.parquet - Fichier vide (0 bytes)
⚠️ test-00002-of-00004.parquet - Fichier vide (0 bytes)
✅ train-00010-of-00126.parquet - 450909097 bytes - 2011 lignes
✅ train-00024-of-00126.parquet - 454814147 bytes - 2011 lignes
⚠️ train-00042-of-00126.parquet - Fichier vide (0 bytes)
⚠️ train-00045-of-00126.parquet - Fichier vide (0 bytes)
✅ train-00031-of-00126.parquet - 451351151 bytes - 2010 lignes
⚠️ train-00029-of-00126.parquet - Fichier vide (0 bytes)
⚠️ train-00088-of-00126.parquet - Fichier vide (0 bytes)
✅ test-00003-of-00004.parquet - 373811260 bytes - 1679 lignes
✅ train-00040-of-00126.parquet - 464720190 bytes - 2010 lignes
✅ train-00058-of-00126.parquet - 454220878 bytes - 2010 lignes
⚠️ train-00067-of-00126.parquet - Fichier vide (0 bytes)
✅ train-00076-of-00126.parquet - 446324155 bytes - 2010 lignes
⚠️ train-00064-of-00126.parquet - Fichier vide (0 bytes)
✅ train-00003-of-00126.parquet - 46

In [13]:
import os
import glob

print("=" * 50)
print("SUPPRESSION DES FICHIERS VIDES")
print("=" * 50)

hc_files = glob.glob('afvoices/human-corrected/*.parquet')
deleted_count = 0

for file in hc_files:
    file_size = os.path.getsize(file)
    if file_size == 0:
        os.remove(file)
        deleted_count += 1
        print(f"🗑️ Supprimé : {os.path.basename(file)}")

print(f"\n✅ {deleted_count} fichiers vides supprimés")
print(f"📊 Fichiers restants : {len(glob.glob('afvoices/human-corrected/*.parquet'))}")


SUPPRESSION DES FICHIERS VIDES
🗑️ Supprimé : train-00023-of-00126.parquet
🗑️ Supprimé : test-00002-of-00004.parquet
🗑️ Supprimé : train-00042-of-00126.parquet
🗑️ Supprimé : train-00045-of-00126.parquet
🗑️ Supprimé : train-00029-of-00126.parquet
🗑️ Supprimé : train-00088-of-00126.parquet
🗑️ Supprimé : train-00067-of-00126.parquet
🗑️ Supprimé : train-00064-of-00126.parquet
🗑️ Supprimé : train-00020-of-00126.parquet
🗑️ Supprimé : train-00035-of-00126.parquet
🗑️ Supprimé : train-00025-of-00126.parquet
🗑️ Supprimé : train-00019-of-00126.parquet
🗑️ Supprimé : train-00015-of-00126.parquet
🗑️ Supprimé : train-00052-of-00126.parquet
🗑️ Supprimé : train-00048-of-00126.parquet
🗑️ Supprimé : train-00086-of-00126.parquet
🗑️ Supprimé : train-00004-of-00126.parquet

✅ 17 fichiers vides supprimés
📊 Fichiers restants : 38


In [14]:
import pandas as pd
import soundfile as sf
import os
from pathlib import Path
import glob
import io # Import io module for BytesIO

# Créer les dossiers nécessaires
output_dir = "tts_dataset"
wavs_dir = os.path.join(output_dir, "wavs")
os.makedirs(wavs_dir, exist_ok=True)

print("=" * 50)
print("PRÉPARATION DES DONNÉES")
print("=" * 50)

# Charger tous les fichiers parquet du dossier human-corrected
hc_files = glob.glob('afvoices/human-corrected/*.parquet')
print(f"\n📊 Nombre de fichiers à traiter : {len(hc_files)}")

# Combiner tous les fichiers
all_data = []
for file in hc_files:
    df = pd.read_parquet(file)
    all_data.append(df)

df_combined = pd.concat(all_data, ignore_index=True)
print(f"📏 Total d'échantillons : {len(df_combined)}")

# Afficher un aperçu  azewrxtcyubio,p;:m
print("\n" + "=" * 50)
print("APERÇU DES DONNÉES COMBINÉES")
print("=" * 50)
print(df_combined.head(3))
print(f"\nColonnes disponibles : {list(df_combined.columns)}")
#Suite
print("\n" + "=" * 50)
print("EXTRACTION DES FICHIERS AUDIO")
print("=" * 50)

# Extraire les fichiers audio dans le dossier wavs/
for idx, row in df_combined.iterrows():
    # Nom du fichier audio
    audio_filename = f"audio_{idx:06d}.wav"
    audio_path = os.path.join(wavs_dir, audio_filename)

    # Extract audio bytes from the 'audio' dictionary
    audio_bytes = row['audio']['bytes']

    # Use soundfile to read the audio data and sampling rate from the bytes
    audio_data, sample_rate = sf.read(io.BytesIO(audio_bytes))

    # Sauvegarder le fichier WAV
    sf.write(audio_path, audio_data, sample_rate)

    if (idx + 1) % 100 == 0:
        print(f"✓ {idx + 1}/{len(df_combined)} fichiers traités")

print(f"\n✅ {len(df_combined)} fichiers audio extraits dans {wavs_dir}")

PRÉPARATION DES DONNÉES

📊 Nombre de fichiers à traiter : 38
📏 Total d'échantillons : 75399

APERÇU DES DONNÉES COMBINÉES
                       text  duration  \
0       A tɛna nafa s'i ma.  1.020000   
1                 Ko anw ko  1.084014   
2  [cs] n'i y'a jateminɛ[?]  1.500000   

                                               audio              label-v1  \
0  {'bytes': b'RIFF \xbf\x02\x00WAVEfmt \x10\x00\...      A tɛna nafasi ma   
1  {'bytes': b'RIFF\xc0u\x01\x00WAVEfmt \x10\x00\...              Ko an ko   
2  {'bytes': b'RIFF\xe0\t\x04\x00WAVEfmt \x10\x00...  Mɛ n'i y'a jate minɛ   

                   label-v2  
0        A tɛna nafa s'i ma  
1                  Ko an ko  
2  [cs] n'i y'a jateminɛ[?]  

Colonnes disponibles : ['text', 'duration', 'audio', 'label-v1', 'label-v2']

EXTRACTION DES FICHIERS AUDIO
✓ 100/75399 fichiers traités
✓ 200/75399 fichiers traités
✓ 300/75399 fichiers traités
✓ 400/75399 fichiers traités
✓ 500/75399 fichiers traités
✓ 600/75399 fichiers trait

In [ ]:
import os
import glob
import io
import pandas as pd
import soundfile as sf
import re # Import regex module for robust cleaning
import string # To get standard punctuation

print("\n" + "=" * 50)
print("CRÉATION DU FICHIER METADATA")
print("=" * 50)

# Créer le fichier metadata au format Coqui TTS
# Format: filename|text|normalized_text
metadata_lines = []

# Define allowed characters from config (as used in config_bambara.json)
# Note: This list should match what's in your TTS config for accurate cleaning
allowed_chars_str = "abcdefghijklmnopqrstuvwxyzàáâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ "
allowed_punctuation_str = "!'(),-.:;? "

# Combine and create a regex pattern for characters to KEEP
# Escape special regex characters in allowed_punctuation_str and allowed_chars_str
escaped_allowed_punctuation = re.escape(allowed_punctuation_str.replace(' ', ''))
escaped_allowed_chars = re.escape(allowed_chars_str.replace(' ', ''))

# The pattern matches any character NOT in our allowed set
allowed_pattern = re.compile(f'[^a-zA-Z0-9{escaped_allowed_chars}{escaped_allowed_punctuation}\s]')


for idx, row in df_combined.iterrows():
    audio_filename_col = f"audio_{idx:06d}"
    #.wav"
    text_raw = str(row['text'])

    # Step 1: Replace any characters not in our allowed set with a space.
    # This is more robust as it filters out unexpected characters, including problematic pipes.
    text_cleaned_for_col = allowed_pattern.sub(' ', text_raw)

    # Step 2: Normalize whitespace (multiple spaces to single space, strip leading/trailing)
    text_cleaned_for_col = ' '.join(text_cleaned_for_col.split()).strip()

    # If the cleaned text is empty, skip this entry to avoid malformed lines
    if not text_cleaned_for_col:
        print(f"⚠️ Skipping audio_{idx:06d}.wav due to empty cleaned text after aggressive cleaning.")
        continue

    # LJSpeech format: filename|text|normalized_text
    # We use the same cleaned text for both for simplicity, but ensures structure.
    metadata_lines.append(f"{audio_filename_col}|{text_cleaned_for_col}|{text_cleaned_for_col}")

# Sauvegarder metadata.csv
metadata_path = os.path.join(output_dir, "metadata.csv")
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"✅ Fichier metadata créé : {metadata_path}")
print(f"📊 Nombre d'entrées : {len(metadata_lines)}")

# Afficher quelques exemples
print("\n📋 Exemples de metadata :")
for line in metadata_lines[:3]:
    print(f"  {line}")

In [ ]:
import os
import glob
import io
import pandas as pd
import soundfile as sf
import re # Import regex module for robust cleaning
import string # To get standard punctuation

print("\n" + "=" * 50)
print("CRICATION DU FICHIER METADATA")
print("=" * 50)

# CrICATION du fichier metadata au format Coqui TTS
# Format: filename|text|normalized_text
metadata_lines = []

# Define allowed characters from config (as used in config_bambara.json)
# These strings explicitly include a space at the end.
allowed_chars_from_config = "abcdefghijklmnopqrstuvwxyz@áâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ "
allowed_punctuation_from_config = "!'(),-.:;? "

# Combine all characters that should be explicitly allowed for cleaning.
# This set includes: lowercase and uppercase English letters, digits, custom Bambara characters,
# specified punctuations, and space.
all_allowed_literal_chars = allowed_chars_from_config + allowed_punctuation_from_config + string.ascii_uppercase + string.digits

# Create a set to remove any duplicate characters and ensure uniqueness.
all_allowed_set = set(all_allowed_literal_chars)
all_allowed_for_regex_str = "".join(sorted(list(all_allowed_set))) # Sort for consistent escaping

# Escape all characters in the combined string for safe use within a regex character set.
escaped_all_allowed = re.escape(all_allowed_for_regex_str)

# The pattern now matches any character NOT in our explicitly defined allowed set.
# This robustly filters out problematic characters (like '|' or unhandled special chars)
# and explicitly includes space, avoiding the SyntaxWarning with '\s'.
allowed_pattern = re.compile(f'[{escaped_all_allowed}]')

# The pattern should be to match characters NOT in the set, so it needs to be inverted.
# Corrected pattern for characters to *remove*:
chars_to_replace_pattern = re.compile(f'[^' + escaped_all_allowed + ']')

for idx, row in df_combined.iterrows():
    audio_filename_col = f"audio_{idx:06d}.wav"
    text_raw = str(row['text'])

    # Step 1: Replace any characters not in our allowed set with a space.
    # This is more robust as it filters out unexpected characters, including problematic pipes.
    text_cleaned_for_col = chars_to_replace_pattern.sub(' ', text_raw)

    # Step 2: Normalize whitespace (multiple spaces to single space, strip leading/trailing)
    text_cleaned_for_col = ' '.join(text_cleaned_for_col.split()).strip()

    # If the cleaned text is empty, skip this entry to avoid malformed lines
    if not text_cleaned_for_col:
        print(f"⚠ï¸ Skipping audio_{idx:06d}.wav due to empty cleaned text after aggressive cleaning.")
        continue

    # LJSpeech format: filename|text|normalized_text
    # We use the same cleaned text for both for simplicity, but ensures structure.
    metadata_lines.append(f"{audio_filename_col}|{text_cleaned_for_col}|{text_cleaned_for_col}")

# Sauvegarder metadata.csv
metadata_path = os.path.join(output_dir, "metadata.csv")
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"✅ Fichier metadata créé : {metadata_path}")
print(f"📊 Nombre d'entrées : {len(metadata_lines)}")

# Afficher quelques exemples
print("\n📋 Exemples de metadata :")
for line in metadata_lines[:3]:
    print(f"  {line}")

In [17]:
print("\n" + "=" * 50)
print("VÉRIFICATION DE LA QUALITÉ")
print("=" * 50)

# Vérifier les durées audio
durations = []
for idx, row in df_combined.iterrows():
    audio_filename = f"audio_{idx:06d}.wav"
    audio_path = os.path.join(wavs_dir, audio_filename)

    info = sf.info(audio_path)
    durations.append(info.duration)

import numpy as np
durations = np.array(durations)

print(f"\n📊 Statistiques des durées audio :")
print(f"  • Durée moyenne : {durations.mean():.2f}s")
print(f"  • Durée min : {durations.min():.2f}s")
print(f"  • Durée max : {durations.max():.2f}s")
print(f"  • Durée totale : {durations.sum()/3600:.2f}h")

# Recommandations
if durations.sum() < 3600:
    print("\n⚠️  Attention : moins d'1h de données (recommandé: 5-10h minimum)")
elif durations.sum() < 18000:
    print("\n✓ Quantité acceptable (1-5h)")
else:
    print("\n✅ Bonne quantité de données (>5h)")


VÉRIFICATION DE LA QUALITÉ

📊 Statistiques des durées audio :
  • Durée moyenne : 2.18s
  • Durée min : 1.02s
  • Durée max : 30.00s
  • Durée totale : 45.71h

✅ Bonne quantité de données (>5h)


In [18]:
from sklearn.model_selection import train_test_split

print("\n" + "=" * 50)
print("DIVISION DES DONNÉES")
print("=" * 50)

# Lire le metadata
with open(metadata_path, 'r', encoding='utf-8') as f:
    # Filter out empty lines or lines with only whitespace
    all_lines = [line for line in f.readlines() if line.strip()]

# Division 80% train, 10% val, 10% test
train_lines, temp_lines = train_test_split(all_lines, test_size=0.2, random_state=42)
val_lines, test_lines = train_test_split(temp_lines, test_size=0.5, random_state=42)

# Sauvegarder les fichiers
with open(os.path.join(output_dir, "metadata_train.csv"), 'w', encoding='utf-8') as f:
    f.writelines(train_lines)

with open(os.path.join(output_dir, "metadata_val.csv"), 'w', encoding='utf-8') as f:
    f.writelines(val_lines)

with open(os.path.join(output_dir, "metadata_test.csv"), 'w', encoding='utf-8') as f:
    f.writelines(test_lines)

print(f"✅ Train : {len(train_lines)} échantillons")
print(f"✅ Val   : {len(val_lines)} échantillons")
print(f"✅ Test  : {len(test_lines)} échantillons")


DIVISION DES DONNÉES
✅ Train : 60319 échantillons
✅ Val   : 7540 échantillons
✅ Test  : 7540 échantillons


In [21]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.3 MB/s  0:00:00


In [25]:
# Créer config.json pour Coqui TTS
import json

config = {
    "model": "tacotron2",
    "run_name": "bambara_tts",
    "run_description": "Modèle TTS pour le Bambara (Mali)",

    # Chemins des données
    "output_path": "output/",
    "datasets": [{
        "name": "bambara",
        "path": "tts_dataset/",
        "meta_file_train": "metadata_train.csv",
        "meta_file_val": "metadata_val.csv",
        "formatter": "ljspeech" # Specify ljspeech formatter as our metadata is in this format
    }],

    # Paramètres audio
    "audio": {
        "sample_rate": 22050,
        "hop_length": 256,
        "win_length": 1024,
        "fft_size": 1024,
        "num_mels": 80,
        "mel_fmin": 0,
        "mel_fmax": 8000
    },

    # Paramètres d'entraînement
    "batch_size": 32,
    "eval_batch_size": 16,
    "num_loader_workers": 4,
    "num_eval_loader_workers": 2,
    "run_eval": True,
    "test_delay_epochs": 10,

    "epochs": 1000,
    "text_cleaner": "basic_cleaners",
    "use_phonemes": False,
    "phoneme_language": "bm",  # Code ISO pour Bambara

    "print_step": 25,
    "save_step": 1000,
    "checkpoint": True,
    "keep_all_best": True,
    "keep_after": 10000,

    # Optimiseur
    "lr": 0.0001,
    "grad_clip": 5.0,

    # Caractères spécifiques au Bambara
    "characters": {
        "characters": "abcdefghijklmnopqrstuvwxyzàáâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ ",
        "punctuations": "!'(),-.:;? ",
        "pad": "_"
    }
}

config_path = "config_bambara.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Configuration sauvegardée : {config_path}")


✅ Configuration sauvegardée : config_bambara.json


In [ ]:
import os
import signal

# Envoyer un signal d'interruption au processus
os.kill(os.getpid(), signal.SIGINT)

In [ ]:
# Script d'entraînement : train_tts.py
from TTS.bin.train_tts import main
import sys
import os

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara.json']
main()


DÉMARRAGE DE L'ENTRAÎNEMENT
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset


INFO:TTS.tts.datasets:Found 60318 files in /content/tts_dataset


Using model: tacotron2
Using model: tacotron2


INFO:TTS.tts.models:Using model: tacotron2


Setting up Audio Processor...
Setting up Audio Processor...


...

INFO:TTS.tts.utils.text.tokenizer:	| use_eos_bos: False


	| use_phonemes: False
	| use_phonemes: False


INFO:TTS.tts.utils.text.tokenizer:	| use_phonemes: False


| Number of instances : 60318
| Number of instances : 60318


INFO:TTS.tts.datasets.dataset:| Number of instances : 60318


# Task
Restart the runtime and run all cells to ensure a clean environment and regenerate the metadata files with proper cleaning, then attempt to train the Coqui TTS model.

## Restart Runtime and Run All Cells

### Subtask:
Restart the Colab runtime and then run all cells from the beginning to ensure that all changes to the metadata generation and cleaning are applied correctly and that no stale files are being used by the Coqui TTS training script.


## Restart Runtime and Run All Cells

### Subtask:
Restart the Colab runtime and then run all cells from the beginning to ensure that all changes to the metadata generation and cleaning are applied correctly and that no stale files are being used by the Coqui TTS training script.

#### Instructions
1. Go to the 'Runtime' menu at the top of the Colab interface.
2. Select 'Restart runtime' and confirm the action when prompted.
3. After the runtime has restarted, go to the 'Runtime' menu again.
4. Select 'Run all' to execute all cells from the beginning, ensuring a clean and consistent state for the environment and data processing.

## Verify Metadata Creation (again)

### Subtask:
Re-run the cell that creates the `metadata.csv` file to ensure the cleaning logic is executed and the file is generated with the correct format for Coqui TTS training.


**Reasoning**:
The subtask requires re-running the cell that creates the `metadata.csv` file to ensure proper text cleaning and formatting. The content of cell `cpJrrw8aRFls` achieves this.



In [42]:
import os
import glob
import io
import pandas as pd
import soundfile as sf
import re # Import regex module for robust cleaning
import string # To get standard punctuation

print("\n" + "=" * 50)
print("CRÉATION DU FICHIER METADATA")
print("=" * 50)

# Créer le fichier metadata au format Coqui TTS
# Format: filename|text|normalized_text
metadata_lines = []

# Define allowed characters from config (as used in config_bambara.json)
# Note: This list should match what's in your TTS config for accurate cleaning
allowed_chars_str = "abcdefghijklmnopqrstuvwxyzàáâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ "
allowed_punctuation_str = "!'(),-.:;? "

# Combine and create a regex pattern for characters to KEEP
# Escape special regex characters in allowed_punctuation_str and allowed_chars_str
escaped_allowed_punctuation = re.escape(allowed_punctuation_str.replace(' ', ''))
escaped_allowed_chars = re.escape(allowed_chars_str.replace(' ', ''))

# The pattern matches any character NOT in our allowed set
allowed_pattern = re.compile(f'[^a-zA-Z0-9{escaped_allowed_chars}{escaped_allowed_punctuation}\s]')


for idx, row in df_combined.iterrows():
    audio_filename_col = f"audio_{idx:06d}.wav"
    text_raw = str(row['text'])

    # Step 1: Replace any characters not in our allowed set with a space.
    # This is more robust as it filters out unexpected characters, including problematic pipes.
    text_cleaned_for_col = allowed_pattern.sub(' ', text_raw)

    # Step 2: Normalize whitespace (multiple spaces to single space, strip leading/trailing)
    text_cleaned_for_col = ' '.join(text_cleaned_for_col.split()).strip()

    # If the cleaned text is empty, skip this entry to avoid malformed lines
    if not text_cleaned_for_col:
        print(f"⚠️ Skipping audio_{idx:06d}.wav due to empty cleaned text after aggressive cleaning.")
        continue

    # LJSpeech format: filename|text|normalized_text
    # We use the same cleaned text for both for simplicity, but ensures structure.
    metadata_lines.append(f"{audio_filename_col}|{text_cleaned_for_col}|{text_cleaned_for_col}")

# Sauvegarder metadata.csv
metadata_path = os.path.join(output_dir, "metadata.csv")
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"✅ Fichier metadata créé : {metadata_path}")
print(f"📊 Nombre d'entrées : {len(metadata_lines)}")

# Afficher quelques exemples
print("\n📋 Exemples de metadata :")
for line in metadata_lines[:3]:
    print(f"  {line}")

<>:28: SyntaxWarning: invalid escape sequence '\s'
<>:28: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2558999285.py:28: SyntaxWarning: invalid escape sequence '\s'
  allowed_pattern = re.compile(f'[^a-zA-Z0-9{escaped_allowed_chars}{escaped_allowed_punctuation}\s]')



CRÉATION DU FICHIER METADATA
✅ Fichier metadata créé : tts_dataset/metadata.csv
📊 Nombre d'entrées : 75399

📋 Exemples de metadata :
  audio_000000.wav|A tɛna nafa s'i ma.|A tɛna nafa s'i ma.
  audio_000001.wav|Ko anw ko|Ko anw ko
  audio_000002.wav|cs n'i y'a jateminɛ ?|cs n'i y'a jateminɛ ?


**Reasoning**:
The previous `SyntaxWarning` for `\s` and the `IndexError` during training indicate issues with the regex pattern for text cleaning and the character set used. The original `replace(' ', '')` unintentionally removed spaces from the allowed character list, and the regex construction was less robust. The updated code builds a comprehensive allowed character set based on the `config_bambara.json` and standard text components (digits, uppercase letters), then correctly escapes and uses this set in the regex to ensure proper cleaning and metadata formatting, preventing `IndexError` by reliably stripping disallowed characters like the pipe `|`.



In [43]:
import os
import glob
import io
import pandas as pd
import soundfile as sf
import re # Import regex module for robust cleaning
import string # To get standard punctuation

print("\n" + "=" * 50)
print("CRICATION DU FICHIER METADATA")
print("=" * 50)

# CrICATION du fichier metadata au format Coqui TTS
# Format: filename|text|normalized_text
metadata_lines = []

# Define allowed characters from config (as used in config_bambara.json)
# These strings explicitly include a space at the end.
allowed_chars_from_config = "abcdefghijklmnopqrstuvwxyz@áâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ "
allowed_punctuation_from_config = "!'(),-.:;? "

# Combine all characters that should be explicitly allowed for cleaning.
# This set includes: lowercase and uppercase English letters, digits, custom Bambara characters,
# specified punctuations, and space.
all_allowed_literal_chars = allowed_chars_from_config + allowed_punctuation_from_config + string.ascii_uppercase + string.digits

# Create a set to remove any duplicate characters and ensure uniqueness.
all_allowed_set = set(all_allowed_literal_chars)
all_allowed_for_regex_str = "".join(sorted(list(all_allowed_set))) # Sort for consistent escaping

# Escape all characters in the combined string for safe use within a regex character set.
escaped_all_allowed = re.escape(all_allowed_for_regex_str)

# The pattern now matches any character NOT in our explicitly defined allowed set.
# This robustly filters out problematic characters (like '|' or unhandled special chars)
# and explicitly includes space, avoiding the SyntaxWarning with '\s'.
allowed_pattern = re.compile(f'[{escaped_all_allowed}]')

# The pattern should be to match characters NOT in the set, so it needs to be inverted.
# Corrected pattern for characters to *remove*:
chars_to_replace_pattern = re.compile(f'[^' + escaped_all_allowed + ']')

for idx, row in df_combined.iterrows():
    audio_filename_col = f"audio_{idx:06d}.wav"
    text_raw = str(row['text'])

    # Step 1: Replace any characters not in our allowed set with a space.
    # This is more robust as it filters out unexpected characters, including problematic pipes.
    text_cleaned_for_col = chars_to_replace_pattern.sub(' ', text_raw)

    # Step 2: Normalize whitespace (multiple spaces to single space, strip leading/trailing)
    text_cleaned_for_col = ' '.join(text_cleaned_for_col.split()).strip()

    # If the cleaned text is empty, skip this entry to avoid malformed lines
    if not text_cleaned_for_col:
        print(f"⚠ï¸ Skipping audio_{idx:06d}.wav due to empty cleaned text after aggressive cleaning.")
        continue

    # LJSpeech format: filename|text|normalized_text
    # We use the same cleaned text for both for simplicity, but ensures structure.
    metadata_lines.append(f"{audio_filename_col}|{text_cleaned_for_col}|{text_cleaned_for_col}")

# Sauvegarder metadata.csv
metadata_path = os.path.join(output_dir, "metadata.csv")
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"✅ Fichier metadata créé : {metadata_path}")
print(f"📊 Nombre d'entrées : {len(metadata_lines)}")

# Afficher quelques exemples
print("\n📋 Exemples de metadata :")
for line in metadata_lines[:3]:
    print(f"  {line}")


CRICATION DU FICHIER METADATA
✅ Fichier metadata créé : tts_dataset/metadata.csv
📊 Nombre d'entrées : 75399

📋 Exemples de metadata :
  audio_000000.wav|A tɛna nafa s'i ma.|A tɛna nafa s'i ma.
  audio_000001.wav|Ko anw ko|Ko anw ko
  audio_000002.wav|cs n'i y'a jateminɛ ?|cs n'i y'a jateminɛ ?


**Reasoning**:
With the metadata file regenerated using the corrected text cleaning logic, the next step is to re-run the training cell. This ensures that the Coqui TTS training script uses the properly formatted metadata, which should resolve the `IndexError` previously encountered.



In [44]:
# Script d'entraînement : train_tts.py
from TTS.bin.train_tts import main
import sys
import os

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara.json']
main()


DÉMARRAGE DE L'ENTRAÎNEMENT


IndexError: LJSpeech format expects 3 pipe-delimited columns in the metadata file.

**Reasoning**:
The training script failed because the split metadata files (`metadata_train.csv`, `metadata_val.csv`, `metadata_test.csv`) were generated from an older, incorrectly formatted `metadata.csv` file. Even though `metadata.csv` was corrected in the previous step, the split files were not regenerated. We need to re-create these split files to reflect the correct metadata format before attempting to train again.



In [45]:
from sklearn.model_selection import train_test_split

print("\n" + "=" * 50)
print("DIVISION DES DONNÉES")
print("=" * 50)

# Lire le metadata
with open(metadata_path, 'r', encoding='utf-8') as f:
    # Filter out empty lines or lines with only whitespace
    all_lines = [line for line in f.readlines() if line.strip()]

# Division 80% train, 10% val, 10% test
train_lines, temp_lines = train_test_split(all_lines, test_size=0.2, random_state=42)
val_lines, test_lines = train_test_split(temp_lines, test_size=0.5, random_state=42)

# Sauvegarder les fichiers
with open(os.path.join(output_dir, "metadata_train.csv"), 'w', encoding='utf-8') as f:
    f.writelines(train_lines)

with open(os.path.join(output_dir, "metadata_val.csv"), 'w', encoding='utf-8') as f:
    f.writelines(val_lines)

with open(os.path.join(output_dir, "metadata_test.csv"), 'w', encoding='utf-8') as f:
    f.writelines(test_lines)

print(f"✅ Train : {len(train_lines)} échantillons")
print(f"✅ Val   : {len(val_lines)} échantillons")
print(f"✅ Test  : {len(test_lines)} échantillons")


DIVISION DES DONNÉES
✅ Train : 60319 échantillons
✅ Val   : 7540 échantillons
✅ Test  : 7540 échantillons


**Reasoning**:
With the split metadata files now correctly regenerated, the next step is to re-attempt the Coqui TTS training. The training script should now be able to parse the metadata files without encountering the previous `IndexError`.



In [46]:
# Script d'entraînement : train_tts.py
from TTS.bin.train_tts import main
import sys
import os

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara.json']
main()


DÉMARRAGE DE L'ENTRAÎNEMENT
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset


INFO:TTS.tts.datasets:Found 60318 files in /content/tts_dataset


Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2


INFO:TTS.tts.models:Using model: tacotron2


Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...


...

 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256
 | hop_length: 256


INFO:TTS.utils.audio.processor: | hop_length: 256


 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024
 | win_length: 1024


INFO:TTS.utils.audio.processor: | win_length: 1024


AssertionError:  [!] There are duplicate characters in the character set. {' '}

**Reasoning**:
The previous error 'AssertionError: [!] There are duplicate characters in the character set. {' '}' indicates that the space character is present in both the 'characters' and 'punctuations' fields of the configuration. To fix this, I will modify the 'config_bambara.json' by removing the space from the 'characters' string, ensuring uniqueness.



In [47]:
import json

config = {
    "model": "tacotron2",
    "run_name": "bambara_tts",
    "run_description": "Modèle TTS pour le Bambara (Mali)",

    # Chemins des données
    "output_path": "output/",
    "datasets": [{
        "name": "bambara",
        "path": "tts_dataset/",
        "meta_file_train": "metadata_train.csv",
        "meta_file_val": "metadata_val.csv",
        "formatter": "ljspeech" # Specify ljspeech formatter as our metadata is in this format
    }],

    # Paramètres audio
    "audio": {
        "sample_rate": 22050,
        "hop_length": 256,
        "win_length": 1024,
        "fft_size": 1024,
        "num_mels": 80,
        "mel_fmin": 0,
        "mel_fmax": 8000
    },

    # Paramètres d'entraînement
    "batch_size": 32,
    "eval_batch_size": 16,
    "num_loader_workers": 4,
    "num_eval_loader_workers": 2,
    "run_eval": True,
    "test_delay_epochs": 10,

    "epochs": 1000,
    "text_cleaner": "basic_cleaners",
    "use_phonemes": False,
    "phoneme_language": "bm",  # Code ISO pour Bambara

    "print_step": 25,
    "save_step": 1000,
    "checkpoint": True,
    "keep_all_best": True,
    "keep_after": 10000,

    # Optimiseur
    "lr": 0.0001,
    "grad_clip": 5.0,

    # Caractères spécifiques au Bambara
    "characters": {
        "characters": "abcdefghijklmnopqrstuvwxyzàáâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ", # Removed trailing space
        "punctuations": "!'(),-.:;? ",
        "pad": "_"
    }
}

config_path = "config_bambara.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Configuration sauvegardée : {config_path}")


✅ Configuration sauvegardée : config_bambara.json


**Reasoning**:
With the `config_bambara.json` file updated to fix the duplicate character error, the next step is to re-attempt the Coqui TTS training. The training script should now be able to initialize the tokenizer without issue.



In [48]:
import sys
import os
from TTS.bin.train_tts import main

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara.json']
main()


DÉMARRAGE DE L'ENTRAÎNEMENT
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset


INFO:TTS.tts.datasets:Found 60318 files in /content/tts_dataset


Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2


INFO:TTS.tts.models:Using model: tacotron2


Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...


...

INFO:TTS.tts.utils.text.tokenizer:	| use_phonemes: False


| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318
| Number of instances : 60318


INFO:TTS.tts.datasets.dataset:| Number of instances : 60318
 ! Run is removed from output/bambara_tts-January-16-2026_10+37AM-0000000
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/trainer/trainer.py", line 688, in get_train_dataloader
    return model.get_train_data_loader(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/trainer/model.py", line 131, in get_train_data_loader
    raise NotImplementedError
NotImplementedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/TTS/tts/datasets/dataset.py", line 53, in _get_audio_size_torchcodec
    from torchcodec.decoders import AudioDecoder
ModuleNotFoundError: No module named 'torchcodec'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/trainer/trainer.py", 

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/trainer/trainer.py", line 688, in get_train_dataloader
    return model.get_train_data_loader(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/trainer/model.py", line 131, in get_train_data_loader
    raise NotImplementedError
NotImplementedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/TTS/tts/datasets/dataset.py", line 53, in _get_audio_size_torchcodec
    from torchcodec.decoders import AudioDecoder
ModuleNotFoundError: No module named 'torchcodec'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/trainer/trainer.py", line 1551, in fit
    self._fit()
  File "/usr/local/lib/python3.12/dist-packages/trainer/trainer.py", line 1503, in _fit
    self.tra

TypeError: object of type 'NoneType' has no len()

**Reasoning**:
The training script failed because of a missing dependency, `torchcodec`. I need to install this library to resolve the `ImportError` and allow the audio processing to proceed.



In [49]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.7 MB/s  0:00:00


**Reasoning**:
With the `torchcodec` library now installed, the next step is to re-attempt the Coqui TTS training. This should resolve the `ImportError` that was preventing the training from starting.



In [1]:
import sys
import os
from TTS.bin.train_tts import main

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara.json']
main()

ModuleNotFoundError: No module named 'TTS'